# Kaggle TGS Salt Identification Challenge U-Net

This notebook provides a simple exploration of the dataset for the Kaggle TGS Salt Identification Challenge

Some code from:
* https://www.kaggle.com/dingli/seismic-data-analysis-with-u-net
* https://www.kaggle.com/shaojiaxin/u-net-with-simple-resnet-blocks-v2-new-loss

## Setup

In [1]:
# Standard python packages
import os
import sys
import datetime

# Other package imports
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.layers import Input, BatchNormalization, Concatenate
from keras.layers.core import Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model, load_model
from keras.optimizers import Adam

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


Setup some global settings and configuration

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
data_folder = os.path.join(project_root, 'data')
data_folder_raw = os.path.join(data_folder, 'raw')
data_folder_submissions = os.path.join(data_folder, 'submissions')
src_folder = os.path.join(project_root, 'src')

train_path = os.path.join(data_folder_raw, 'train')
train_images_path = os.path.join(train_path, 'images')
train_masks_path = os.path.join(train_path, 'masks')
train_file = os.path.join(data_folder_raw, 'train.csv')

test_path = os.path.join(data_folder_raw, 'test')
test_images_path = os.path.join(test_path, 'images')

depth_file = os.path.join(data_folder_raw, 'depths.csv')

# Other parameters / shared functions
img_size_ori = 101
# UNet image size
#img_size_target = 128
# UNetResnet image size
img_size_target = 101

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]

This notebook uses the shared package however first we need to ensure it is available (otherwise you get an error about the module not being found). You can either run setup.py as discussed in the readme to install the package or modify the path to include the src folder.

In [3]:
# Explicitly set path so don't need to run setup.py - if we have multiple copies of 
# the code we would otherwise need to setup a seperate environment for each to
# ensure the code pointers are correct.
sys.path.insert(0, src_folder)

from tgssalt_challenge.submission import rle_encode
from tgssalt_challenge.scoring import iou_metric, iou_metric_batch, my_iou_metric
from tgssalt_challenge.unet import UNet, UNetResNetBlocks

## Load data
Look at the train file that contains image id's along with a mask of salt regions

In [4]:
train_df = pd.read_csv(train_file, index_col="id", usecols=[0])
depths_df = pd.read_csv(depth_file, index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

In [5]:
train_df["images"] = [np.array(load_img(os.path.join(train_images_path, "{}.png").format(idx), color_mode="grayscale")) / 255 for idx in tqdm_notebook(train_df.index)]

In [6]:
train_df["masks"] = [np.array(load_img(os.path.join(train_masks_path, "{}.png").format(idx), color_mode="grayscale")) / 255 for idx in tqdm_notebook(train_df.index)]

## Calculating the salt coverage and salt coverage classes
Counting the number of salt pixels in the masks and dividing them by the image size. Also create 11 coverage classes, -0.1 having no salt at all to 1.0 being salt only. Plotting the distribution of coverages and coverage classes, and the class against the raw coverage.

In [7]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

In [8]:
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

## Create train/validation split stratified by salt coverage
Using the salt coverage as a stratification criterion. Also show an image to check for correct upsampling.

In [9]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state=1337)

## Data augmentation
Add a flipped version of the images

In [10]:
x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)

## Build Model
Chose which of the below we want to run
### Basic UNet
Note - set the image size near the top to 128

In [ ]:
model = UNet((img_size_target,img_size_target,1),start_ch=16,depth=5,batchnorm=True)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model_name = "./U-Net-keras.model"

### Resnet UNet
Note - set the image size near the top to 101

In [12]:
model = UNetResNetBlocks(img_size_target)
#c = optimizers.adam(lr = 0.01)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[my_iou_metric])

model_name = "./U-Net-ResNet-keras.model"

### Model info

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 101, 101, 16) 160         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 101, 101, 16) 64          conv2d_20[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 101, 101, 16) 0           batch_normalization_17[0][0]     
__________________________________________________________________________________________________
conv2d_21 

In [14]:
print(x_train.shape)

(6400, 101, 101, 1)


## Training

In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint(model_name, save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000005, verbose=1)
tf_callback = TensorBoard(log_dir='./logdir', histogram_freq=0, write_graph=True, write_images=True)

epochs = 1000
batch_size = 32

history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr, tf_callback],
                    verbose=1)

Train on 6400 samples, validate on 800 samples
Epoch 1/1000
6400/6400 [==============================] - 68s 11ms/step - loss: 0.5449 - my_iou_metric: 0.3027 - val_loss: 0.5013 - val_my_iou_metric: 0.4266

Epoch 00001: val_loss improved from inf to 0.50132, saving model to ./U-Net-ResNet-keras.model
Epoch 2/1000
6400/6400 [==============================] - 49s 8ms/step - loss: 0.3460 - my_iou_metric: 0.5133 - val_loss: 4.1821 - val_my_iou_metric: 0.0598

Epoch 00002: val_loss did not improve from 0.50132
Epoch 3/1000
6400/6400 [==============================] - 49s 8ms/step - loss: 0.2727 - my_iou_metric: 0.5511 - val_loss: 0.3648 - val_my_iou_metric: 0.3930

Epoch 00003: val_loss improved from 0.50132 to 0.36475, saving model to ./U-Net-ResNet-keras.model
Epoch 4/1000
6400/6400 [==============================] - 49s 8ms/step - loss: 0.2406 - my_iou_metric: 0.5776 - val_loss: 0.2519 - val_my_iou_metric: 0.5649

Epoch 00004: val_loss improved from 0.36475 to 0.25194, saving model to ./U

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax_loss.legend()
ax_acc.plot(history.epoch, history.history["acc"], label="Train accuracy")
ax_acc.plot(history.epoch, history.history["val_acc"], label="Validation accuracy")
ax_acc.legend()

## Predict the validation set

In [ ]:
model = load_model(model_name)

In [ ]:
preds_valid = model.predict(x_valid).reshape(-1, img_size_target, img_size_target)
preds_valid = np.array([downsample(x) for x in preds_valid])
mask_valid = np.array([downsample(x) for x in y_valid])

### Scoring
The output mask contains real numbers between 0 and 1 however we need to convert these to binary values to represent either off or on. We evaluate different thresholds to find the level that gives the best IoU score.

In [ ]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(mask_valid, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
print("Best Threshold: ", threshold_best)
print("Best IoU: ", iou_best)

In [ ]:
plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

Convert to a binary mask based upon the calculated best threshold

In [ ]:
for i, idx in enumerate(preds_valid):
    preds_valid[i] = np.int32(preds_valid[i] > threshold_best)

## Sanity check with adjusted threshold
Again some sample images with the adjusted threshold.

In [ ]:
max_images = 30
grid_width = 5
grid_height = int(max_images / grid_width)*3
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*4, grid_height*4))
for i, idx in enumerate(ids_valid[:max_images]):
    #print(idx)
    img = downsample(np.squeeze(x_valid[i]))
    mask = np.squeeze(mask_valid[i])
    pred = np.squeeze(preds_valid[i])
    ax_image = axs[int(i / grid_width)*3, i % grid_width]
    ax_image.imshow(img, cmap="Greys")
    ax_image.set_title("Image {0}\nDepth: {1}".format(idx, train_df.loc[idx].z))
    ax_image.set_yticklabels([])
    ax_image.set_xticklabels([])
    ax_mask = axs[int(i / grid_width)*3+1, i % grid_width]
    ax_mask.imshow(img, cmap="Greys")
    ax_mask.imshow(mask, alpha=0.9, cmap="Greens")
    ax_mask.set_title("Mask {0}\nCoverage: {1}".format(idx,  round(train_df.loc[idx].coverage, 2)))
    ax_mask.set_yticklabels([])
    ax_mask.set_xticklabels([])
    ax_pred = axs[int(i / grid_width)*3+2, i % grid_width]
    ax_pred.imshow(img, cmap="Greys")
    ax_pred.imshow(pred, alpha=0.9, cmap="Blues")
    coverage_pred = np.sum(pred) / pow(img_size_ori, 2)
    ax_pred.set_title("Predict {0}\nCoverage: {1}".format(idx,  round(coverage_pred, 2)))
    ax_pred.set_yticklabels([])
    ax_pred.set_xticklabels([])

In [ ]:
# plot small charts
max_images = 24
grid_width = 12
grid_height = int(max_images / grid_width)*3
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*2, grid_height*2))
for i, idx in enumerate(ids_valid[:max_images]):
    #print(idx)
    img = downsample(np.squeeze(x_valid[i]))
    mask = np.squeeze(mask_valid[i])
    pred = np.squeeze(preds_valid[i])
    ax_image = axs[int(i / grid_width)*3, i % grid_width]
    ax_image.imshow(img, cmap="Greys")
    ax_image.set_title("Image")
    ax_image.set_yticklabels([])
    ax_image.set_xticklabels([])
    ax_mask = axs[int(i / grid_width)*3+1, i % grid_width]
    ax_mask.imshow(img, cmap="Greys")
    ax_mask.imshow(mask, alpha=0.9, cmap="Greens")
    ax_mask.set_title("Mask")
    ax_mask.set_yticklabels([])
    ax_mask.set_xticklabels([])
    ax_pred = axs[int(i / grid_width)*3+2, i % grid_width]
    ax_pred.imshow(img, cmap="Greys")
    ax_pred.imshow(pred, alpha=0.9, cmap="Blues")
    coverage_pred = np.sum(pred) / pow(img_size_ori, 2)
    ax_pred.set_title("Predict")
    ax_pred.set_yticklabels([])
    ax_pred.set_xticklabels([])

### Check effect of clearing masks with only a small amount of predictions
Here we can see that clearing masks with less than approximately 80 pixels might help.

Test with 80 reduced score from 0.758 to 0.757

Test with 50 reduced score from 0.758 to 0.755

Perhaps this isn't helping much! Leaving at a low value now pending further testing!

We might be best looking at filling / clearing based upon a sliding window.

In [ ]:
print('origin iou:', iou_metric_batch(mask_valid,preds_valid))
for pixel_thres in [5,10,20,30,40,50,60,70,80,90,100,200,300, 500, 800, 1000]:
    preds_valid_final_copy = np.copy(preds_valid)
    for index,img in enumerate(preds_valid_final_copy):
        pixel_count=np.sum(img)
        if pixel_count and pixel_count < pixel_thres:
            # print(pixel_count)
            preds_valid_final_copy[index]=np.zeros([101,101])
    print('zero out iou for mask under '+str(pixel_thres)+' pixels: '+str(iou_metric_batch(mask_valid,preds_valid_final_copy)))
    
best_minimum_mask_count = 20

## Submission
Load, predict and submit the test image predictions.

First load the images into an array

In [ ]:
x_test = np.array([upsample(np.array(load_img(os.path.join(test_images_path, "{}.png").format(idx), color_mode = "grayscale"))) / 255 for idx in tqdm_notebook(test_df.index)]).reshape(-1, img_size_target, img_size_target, 1)

Predict the mask and convert to binary

In [ ]:
preds_test = model.predict(x_test)

In [ ]:
preds_test_final = np.array([downsample(x) for x in preds_test])
for i, idx in enumerate(preds_test_final):
    preds_test_final[i] = np.int32(preds_test_final[i] > threshold_best)

Clear any masks with less than the calculated minimum pixels

In [ ]:
for index,img in enumerate(preds_test_final):
    pixel_count=np.sum(img)
    if pixel_count and pixel_count < best_minimum_mask_count:
        # print(pixel_count)
        preds_test_final[index]=np.zeros([101,101,1])

Create the final data frame and submission file

In [ ]:
pred_dict = {idx: rle_encode(preds_test_final[i]) for i, idx in enumerate(tqdm_notebook(test_df.index.values))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv(os.path.join(data_folder_submissions, 'submission.csv'))
output_file = "sub-{}.csv".format(datetime.datetime.now().strftime("%Y%m%d-%H%M"))
sub.to_csv(os.path.join(data_folder_submissions, output_file))
print('Submission output to:', output_file)

## Appendix 1 - Environment Configuration

In [ ]:
print (os.getcwd())
print (sys.version)
print (sys.executable)
print (sys.path)